In [1]:
###### Imports

import pydot
from IPython.display import SVG, display
import pydrake.all
from pydrake.all import (DiagramBuilder, MeshcatVisualizer, Simulator,
                         StartMeshcat)

from manipulation.scenarios import MakeManipulationStation, AddIiwa, AddRgbdSensors, AddRgbdSensor, AddWsg

import numpy as np
from pydrake.all import (AbstractValue, AddMultibodyPlantSceneGraph, Box,
                         Capsule, ContactResults, DiagramBuilder,
                         InverseDynamicsController, JointSliders, LeafSystem,
                         MeshcatVisualizer, MeshcatVisualizerParams,
                         MultibodyPlant, Rgba, RigidTransform, RollPitchYaw,
                         RotationMatrix, Simulator, Sphere, StartMeshcat,
                         StateInterpolatorWithDiscreteDerivative, Parser, LoadModelDirectivesFromString)

from manipulation import running_as_notebook
from manipulation.scenarios import AddFloatingXyzJoint, AddShape

from pydrake.all import (
    AbstractValue, Adder, AddMultibodyPlantSceneGraph, BallRpyJoint, BaseField,
    Box, CameraInfo, ClippingRange, CoulombFriction, Cylinder, Demultiplexer,
    DepthImageToPointCloud, DepthRange, DepthRenderCamera, DiagramBuilder,
    FindResourceOrThrow, GeometryInstance, InverseDynamicsController,
    LeafSystem, LoadModelDirectivesFromString,
    MakeMultibodyStateToWsgStateSystem, MakePhongIllustrationProperties,
    MakeRenderEngineVtk, ModelInstanceIndex, MultibodyPlant, Parser,
    PassThrough, PrismaticJoint, ProcessModelDirectives, RenderCameraCore,
    RenderEngineVtkParams, RevoluteJoint, Rgba, RgbdSensor, RigidTransform,
    RollPitchYaw, RotationMatrix, SchunkWsgPositionController, SpatialInertia,
    Sphere, StateInterpolatorWithDiscreteDerivative, UnitInertia)
from pydrake.manipulation.planner import (
    DifferentialInverseKinematicsIntegrator,
    DifferentialInverseKinematicsParameters)

from manipulation.utils import AddPackagePaths, FindResource

from pydrake.all import (DiagramBuilder, Integrator, JacobianWrtVariable,
                         LeafSystem, MeshcatVisualizer, Simulator,
                         StartMeshcat)

from manipulation.meshcat_utils import AddMeshcatTriad

from pydrake.all import (AbstractValue, ConstantVectorSource, DiagramBuilder,
                         LeafSystem, MeshcatVisualizer,
                         MeshcatVisualizerParams, PiecewisePose,
                         RigidTransform, RotationMatrix, Simulator,
                         StartMeshcat, TrajectorySource)
                         
from manipulation.scenarios import (AddIiwaDifferentialIK,
                                    MakeManipulationStation, AddMultibodyTriad)

from pydrake.all import LeafSystem, PiecewisePolynomial, DiagramBuilder, Simulator

In [2]:
# Start the visualizer.
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at https://ba8e65c7-ba08-42d3-b249-70b2fa081e19.deepnoteproject.com/7001/


In [3]:
# Set-up and simulation

model_directives = """
directives:
- add_model:
    name: iiwa0
    file: package://drake/manipulation/models/iiwa_description/iiwa7/iiwa7_no_collision.sdf
    default_joint_positions:
        iiwa_joint_1: [0]
        iiwa_joint_2: [0.1]
        iiwa_joint_3: [0]
        iiwa_joint_4: [-1.2]
        iiwa_joint_5: [0]
        iiwa_joint_6: [ 1.6]
        iiwa_joint_7: [0]
- add_weld:
    parent: world
    child: iiwa0::iiwa_link_0
    X_PC:
        translation: [-0.5, 0, 0]
        rotation: !Rpy { deg: [0, 0, 90]}
- add_model:
    name: wsg0
    file: package://drake/manipulation/models/wsg_50_description/sdf/schunk_wsg_50_with_tip.sdf
- add_weld:
    parent: iiwa0::iiwa_link_7
    child: wsg0::body
    X_PC:
        translation: [0, 0, 0.09]
        rotation: !Rpy { deg: [90, 0, 90]}
- add_model:
    name: iiwa1
    file: package://drake/manipulation/models/iiwa_description/iiwa7/iiwa7_no_collision.sdf
    default_joint_positions:
        iiwa_joint_1: [0]
        iiwa_joint_2: [0.1]
        iiwa_joint_3: [0]
        iiwa_joint_4: [-1.2]
        iiwa_joint_5: [0]
        iiwa_joint_6: [ 1.6]
        iiwa_joint_7: [0]
- add_weld:
    parent: world
    child: iiwa1::iiwa_link_0
    X_PC:
        translation: [0.5, 0, 0]
        rotation: !Rpy { deg: [0, 0, -90]}
- add_model:
    name: wsg1
    file: package://drake/manipulation/models/wsg_50_description/sdf/schunk_wsg_50_with_tip.sdf
- add_weld:
    parent: iiwa1::iiwa_link_7
    child: wsg1::body
    X_PC:
        translation: [0, 0, 0.09]
        rotation: !Rpy { deg: [90, 0, 90]}

"""

In [4]:
# Writing system
class MeshcatWriter(LeafSystem):

    def __init__(self, meshcat, drawing_body_index, canvas_body_index,
                    rgba, line_width=0.01, draw_threshold=0.01):
        LeafSystem.__init__(self)
        self._meshcat = meshcat
        self._drawing_body_index = drawing_body_index
        self._canvas_body_index = canvas_body_index
        self._rgba = rgba
        self._line_width = line_width
        self._draw_threshold = draw_threshold

        self.DeclareAbstractInputPort("contact_results",
                                        AbstractValue.Make(ContactResults()))
        self.DeclarePeriodicDiscreteUpdateEvent(0.01, 0, self.MaybeDraw)

        self._p_WLastDraw_index = self.DeclareDiscreteState(3)
        self._was_in_contact_index = self.DeclareDiscreteState([0])
        self._num_drawn_index = self.DeclareDiscreteState([0])

    def MaybeDraw(self, context, discrete_state):
        results = self.get_input_port().Eval(context)

        # The point on the canvas that most deeply penetrates the chalk.
        p_WDraw = None
        for i in range(results.num_point_pair_contacts()):
            info = results.point_pair_contact_info(i)
            if (info.bodyA_index() == self._drawing_body_index and
                    info.bodyB_index() == self._canvas_body_index):
                p_WDraw = info.point_pair().p_WCb
                break
            elif (info.bodyB_index() == self._drawing_body_index and
                    info.bodyA_index() == self._canvas_body_index):
                p_WDraw = info.point_pair().p_WCa
                break

        if p_WDraw is not None:
            p_WLastDraw = context.get_discrete_state(
                self._p_WLastDraw_index).get_value()
            was_in_contact = context.get_discrete_state(
                self._was_in_contact_index)[0]
            num_drawn = context.get_discrete_state(self._num_drawn_index)[0]

            length = np.linalg.norm(p_WDraw - p_WLastDraw)
            if (was_in_contact and length > self._draw_threshold):
                meshcat.SetObject(f"writer/{num_drawn}",
                                    Capsule(self._line_width, length),
                                    self._rgba)
                p_WMidpoint = (p_WDraw + p_WLastDraw)/2
                X_WCapsule = RigidTransform(
                    RotationMatrix.MakeFromOneVector(
                        p_WDraw - p_WLastDraw, 2), p_WMidpoint)
                meshcat.SetTransform(f"writer/{num_drawn}", X_WCapsule)
                discrete_state.set_value(self._num_drawn_index,
                                            [num_drawn + 1])
                discrete_state.set_value(self._p_WLastDraw_index, p_WDraw)
            elif not was_in_contact:
                discrete_state.set_value(self._p_WLastDraw_index, p_WDraw)

            discrete_state.set_value(self._was_in_contact_index, [1])
        else:
            discrete_state.set_value(self._was_in_contact_index, [0])

def ChalkController(time_step = 0.002):
    controller_plant = MultibodyPlant(time_step)
    AddShape(controller_plant, Capsule(0.01, 0.2), "chalk", mass=1, mu=1,
        color=[1, .34, .2, 1.0])
    AddFloatingXyzJoint(controller_plant,
                controller_plant.GetFrameByName("chalk"),
                controller_plant.GetModelInstanceByName("chalk"),
                actuators=True)

    controller_plant.Finalize()

    return controller_plant

In [5]:
def AddIiwas(plant):
    parser = Parser(plant)
    AddPackagePaths(parser)
    if model_directives:
        directives = LoadModelDirectivesFromString(model_directives)
        ProcessModelDirectives(directives, parser)

def AddEnvironment(plant):
    # add table
    AddShape(plant, Box(0.8, 1, .2), "table", color=[0.05, 0.05, 0.05, 1.0])
    plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("table"),
                        RigidTransform([0, 0, .12]))

    # add chalk
    chalk_instance = AddShape(plant, Capsule(0.01, 0.2), "chalk", mass=1, mu=1,
            color=[1.0, 0.34, 0.2, 1.0])
    gripper0 = plant.GetModelInstanceByName("wsg0")
    plant.WeldFrames(plant.GetFrameByName("body", gripper0), plant.GetFrameByName("chalk"),
                        RigidTransform([0, 0.075, 0]))

    # add eraser
    eraser_instance = AddShape(plant, Box(0.05, 0.025, 0.125), "eraser", mass=1, mu=1,
            color=[1.0, 1.0, 1.0, 1.0])
    gripper1 = plant.GetModelInstanceByName("wsg1")
    plant.WeldFrames(plant.GetFrameByName("body", gripper1), plant.GetFrameByName("eraser"),
                        RigidTransform([0, 0.125, 0]))
    '''AddFloatingXyzJoint(plant, #[cc]
                        plant.GetFrameByName("chalk"),
                        chalk_instance,
                        actuators=True)''' #Add if we need to have
    return chalk_instance 

def IiwaControllerSetup(builder, plant, scene_graph, time_step = 0.002):
    iiwa_prefix = "iiwa"
    wsg_prefix = "wsg"

    for i in range(plant.num_model_instances()):
        model_instance = ModelInstanceIndex(i)
        model_instance_name = plant.GetModelInstanceName(model_instance)

        if model_instance_name.startswith(iiwa_prefix):
            num_iiwa_positions = plant.num_positions(model_instance)

            # I need a PassThrough system so that I can export the input port.
            iiwa_position = builder.AddSystem(PassThrough(num_iiwa_positions))
            builder.ExportInput(iiwa_position.get_input_port(),
                                model_instance_name + "_position")
            builder.ExportOutput(iiwa_position.get_output_port(),
                                 model_instance_name + "_position_command")

            # Export the iiwa "state" outputs.
            demux = builder.AddSystem(
                Demultiplexer(2 * num_iiwa_positions, num_iiwa_positions))
            builder.Connect(plant.get_state_output_port(model_instance),
                            demux.get_input_port())
            builder.ExportOutput(demux.get_output_port(0),
                                 model_instance_name + "_position_measured")
            builder.ExportOutput(demux.get_output_port(1),
                                 model_instance_name + "_velocity_estimated")
            builder.ExportOutput(plant.get_state_output_port(model_instance),
                                 model_instance_name + "_state_estimated")

            # Make the plant for the iiwa controller to use.
            controller_plant = MultibodyPlant(time_step=time_step)
            # TODO: Add the correct IIWA model (introspected from MBP)
            if plant.num_positions(model_instance) == 3:
                controller_iiwa = AddPlanarIiwa(controller_plant)
            else:
                controller_iiwa = AddIiwa(controller_plant)
            AddWsg(controller_plant, controller_iiwa, welded=True)
            controller_plant.Finalize()

            # Add the iiwa controller
            iiwa_controller = builder.AddSystem(
                InverseDynamicsController(controller_plant,
                                          kp=[100] * num_iiwa_positions,
                                          ki=[1] * num_iiwa_positions,
                                          kd=[20] * num_iiwa_positions,
                                          has_reference_acceleration=False))
            iiwa_controller.set_name(model_instance_name + "_controller")
            builder.Connect(plant.get_state_output_port(model_instance),
                            iiwa_controller.get_input_port_estimated_state())

            # Add in the feed-forward torque
            adder = builder.AddSystem(Adder(2, num_iiwa_positions))
            builder.Connect(iiwa_controller.get_output_port_control(),
                            adder.get_input_port(0))

            # Use a PassThrough to make the port optional (it will provide zero
            # values if not connected).
            torque_passthrough = builder.AddSystem(
                PassThrough([0] * num_iiwa_positions))
            builder.Connect(torque_passthrough.get_output_port(),
                            adder.get_input_port(1))
            builder.ExportInput(torque_passthrough.get_input_port(),
                                model_instance_name + "_feedforward_torque")
            builder.Connect(adder.get_output_port(),
                            plant.get_actuation_input_port(model_instance))

            # Add discrete derivative to command velocities.
            desired_state_from_position = builder.AddSystem(
                StateInterpolatorWithDiscreteDerivative(
                    num_iiwa_positions,
                    time_step,
                    suppress_initial_transient=True))
            desired_state_from_position.set_name(
                model_instance_name + "_desired_state_from_position")
            builder.Connect(desired_state_from_position.get_output_port(),
                            iiwa_controller.get_input_port_desired_state())
            builder.Connect(iiwa_position.get_output_port(),
                            desired_state_from_position.get_input_port())

            # Export commanded torques.
            builder.ExportOutput(adder.get_output_port(),
                                 model_instance_name + "_torque_commanded")
            builder.ExportOutput(adder.get_output_port(),
                                 model_instance_name + "_torque_measured")

            builder.ExportOutput(
                plant.get_generalized_contact_forces_output_port(
                    model_instance), model_instance_name + "_torque_external")

        elif model_instance_name.startswith(wsg_prefix):

            # Wsg controller.
            wsg_controller = builder.AddSystem(SchunkWsgPositionController())
            wsg_controller.set_name(model_instance_name + "_controller")
            builder.Connect(wsg_controller.get_generalized_force_output_port(),
                            plant.get_actuation_input_port(model_instance))
            builder.Connect(plant.get_state_output_port(model_instance),
                            wsg_controller.get_state_input_port())
            builder.ExportInput(
                wsg_controller.get_desired_position_input_port(),
                model_instance_name + "_position")
            builder.ExportInput(wsg_controller.get_force_limit_input_port(),
                                model_instance_name + "_force_limit")
            wsg_mbp_state_to_wsg_state = builder.AddSystem(
                MakeMultibodyStateToWsgStateSystem())
            builder.Connect(plant.get_state_output_port(model_instance),
                            wsg_mbp_state_to_wsg_state.get_input_port())
            builder.ExportOutput(wsg_mbp_state_to_wsg_state.get_output_port(),
                                 model_instance_name + "_state_measured")
            builder.ExportOutput(wsg_controller.get_grip_force_output_port(),
                                 model_instance_name + "_force_measured")

class Setup():
    def __init__(self, time_step = 0.002):
        self.builder = DiagramBuilder()
        self.plant, self.scene_graph = AddMultibodyPlantSceneGraph(self.builder, time_step = time_step)

        AddIiwas(self.plant) # load in two Iiwa's from model directives
        chalk_instance = AddEnvironment(self.plant)
        self.plant.Finalize() #finalize plant
        self.chalk_instance = chalk_instance

        '''chalk_controller_plant = ChalkController()
        self.chalk_controller_plant = chalk_controller_plant
        self.chalk_instance = chalk_instance
#
        controller = self.builder.AddSystem(
            InverseDynamicsController(self.chalk_controller_plant,
                                    kp=[20] * 3,
                                    ki=[0] * 3,
                                    kd=[10] * 3,
                                    has_reference_acceleration=False))
        #set_trace()

        self.builder.Connect(self.plant.get_state_output_port(self.chalk_instance),
                        controller.get_input_port_estimated_state())
        self.builder.Connect(controller.get_output_port(),
                        self.plant.get_actuation_input_port(self.chalk_instance))'''

        IiwaControllerSetup(self.builder, self.plant, self.scene_graph, time_step = time_step)

        self.builder.ExportOutput(self.scene_graph.get_query_output_port(), "geometry_query")
        self.builder.ExportOutput(self.plant.get_contact_results_output_port(), 
                            "contact_results")
        self.builder.ExportOutput(self.plant.get_state_output_port(), 
                            "plant_continuous_state")
    def build_diagram(self):
        diagram = self.builder.Build()
        self.diagram = diagram
        return diagram

class PseudoInverseController(LeafSystem):
    def __init__(self, plant, number = "0"):
        LeafSystem.__init__(self)
        self._plant = plant
        self._plant_context = plant.CreateDefaultContext()
        self._iiwa = plant.GetModelInstanceByName("iiwa"+number)
        self._G = plant.GetBodyByName("body", plant.GetModelInstanceByName("wsg"+number)).body_frame()
        self._W = plant.world_frame()

        self.DeclareVectorInputPort("iiwa"+number+"_position", 7)
        self.DeclareVectorOutputPort("iiwa"+number+"_velocity", 7,
                                     self.CalcOutput)

    def CalcOutput(self, context, output):
        q = self.get_input_port().Eval(context)
        self._plant.SetPositions(self._plant_context, self._iiwa, q)
        J_G = self._plant.CalcJacobianSpatialVelocity(
            self._plant_context, JacobianWrtVariable.kQDot,
            self._G, [0,0,0], self._W, self._W)
        J_G = J_G[:,0:7] # Ignore gripper terms

        V_G_desired = np.array([0,    # rotation about x
                                -.1,  # rotation about y
                                0,    # rotation about z
                                0.05,    # x
                                -.05, # y
                                -.1]) # z
        v = np.linalg.pinv(J_G).dot(V_G_desired)
        output.SetFromVector(v)

def MescatViz(meshcat, iiwa0_traj=None, iiwa1_traj=None):
    builder = DiagramBuilder()
    setup = Setup() ## TODO Create scenario for two robotic arms
    station = builder.AddSystem(setup.build_diagram())
    
    plant = station.GetSubsystemByName("plant")

#    AddMultibodyTriad(
#      station.GetSystems()[0].GetFrameByName("body", station.GetSystems()[0].GetModelInstanceByName('wsg0')), station.GetSystems()[1])
#    AddMultibodyTriad(
#      station.GetSystems()[0].GetFrameByName("body", station.GetSystems()[0].GetModelInstanceByName('wsg1')), station.GetSystems()[1])

    controller_plant = station.GetSubsystemByName(
        "iiwa0_controller").get_multibody_plant_for_control()

    if iiwa0_traj is not None:
        iiwa0_traj_source = builder.AddSystem(PoseTrajectorySource(iiwa0_traj))
        controller = AddIiwaDifferentialIK(
                    builder,
                    controller_plant,
                    frame=controller_plant.GetFrameByName("body"))
        builder.Connect(iiwa0_traj_source.get_output_port(),
            controller.get_input_port(0))
        builder.Connect(station.GetOutputPort("iiwa0_state_estimated"),
            controller.GetInputPort("robot_state"))
        builder.Connect(controller.get_output_port(),
            station.GetInputPort("iiwa0_position"))

    controller_plant2 = station.GetSubsystemByName(
        "iiwa1_controller").get_multibody_plant_for_control()

    if iiwa1_traj is not None:
        iiwa1_traj_source = builder.AddSystem(PoseTrajectorySource(iiwa1_traj))
        controller2 = AddIiwaDifferentialIK(
                    builder,
                    controller_plant2,
                    frame=controller_plant2.GetFrameByName("body"))
        builder.Connect(iiwa1_traj_source.get_output_port(),
            controller2.get_input_port(0))
        builder.Connect(station.GetOutputPort("iiwa1_state_estimated"),
            controller2.GetInputPort("robot_state"))
        builder.Connect(controller2.get_output_port(),
            station.GetInputPort("iiwa1_position"))

    #ADD Chalk Controller #[cc]
    controller_plant_chalk = MultibodyPlant(0.002)
    AddShape(controller_plant_chalk, Capsule(0.01, 0.2), "chalk", mass=1, mu=1,
        color=[1, .34, .2, 1.0])
    controller_plant_chalk.Finalize()

    """controller3 = builder.AddSystem(
        InverseDynamicsController(controller_plant_chalk,
                                  kp=[20] * 3,
                                  ki=[0] * 3,
                                  kd=[10] * 3,
                                  has_reference_acceleration=False))
    builder.Connect(plant.get_state_output_port(setup.chalk_instance),
                    controller3.get_input_port_estimated_state())
    builder.Connect(controller3.get_output_port(),
                    plant.get_actuation_input_port(setup.chalk_instance))"""
    writer = builder.AddSystem(
        MeshcatWriter(meshcat,
                      plant.GetBodyByName("chalk").index(),
                      plant.GetBodyByName("table").index(),
                      Rgba(1.0, 0.34, 0.2, 1.0), line_width=0.005))
    builder.Connect(
        station.GetOutputPort("contact_results"),
                    writer.get_input_port())

    eraser = builder.AddSystem(
        MeshcatWriter(meshcat,
                      plant.GetBodyByName("eraser").index(),
                      plant.GetBodyByName("table").index(),
                      Rgba(0.05, 0.05, 0.05, 1.0), line_width=0.0005))
    builder.Connect(
        station.GetOutputPort("contact_results"),
                    eraser.get_input_port())

    params = MeshcatVisualizerParams()
    params.delete_on_initialization_event = False
    visualizer = MeshcatVisualizer.AddToBuilder(
        builder, station.GetOutputPort("geometry_query"), meshcat, params)

    diagram = builder.Build()
    simulator = Simulator(diagram)
    context = simulator.get_mutable_context()
    station_context = station.GetMyMutableContextFromRoot(context)

    #plant = station.GetSubsystemByName("plant")
    plant_context = plant.GetMyMutableContextFromRoot(context)

    q0 = plant.GetPositions(plant_context, plant.GetModelInstanceByName('iiwa0'))
    q1 = plant.GetPositions(plant_context, plant.GetModelInstanceByName('iiwa1'))
    #station.GetInputPort('iiwa0_position').FixValue(station_context, q0)
    #station.GetInputPort('iiwa1_position').FixValue(station_context, q0)

    station.GetInputPort("wsg0_position").FixValue(station_context,[0])
    station.GetInputPort("wsg1_position").FixValue(station_context,[0])

    #visualizer.StartRecording(False)

    if iiwa1_traj is not None:
        if running_as_notebook:
            while simulator.get_context().get_time() < traj_time + iiwa1_delay_time + 0.1:
                simulator.AdvanceTo(simulator.get_context().get_time() + 0.05)

    #visualizer.PublishRecording()

    pydot.graph_from_dot_data(diagram.GetGraphvizString())[0].write_svg("diagram.svg")

    return diagram, station

In [6]:
#Robot moving

#iiwa0 transforms
p_iiwa0world = [0, -0.5, 0]
p_worldiiwa0 = [-0.5, 0, 0]
p_wsg0chalk = np.array([0, 0.075, 0])
p_chalktable = np.array([0, 0, 0.14])

R_worldwsg0 = RotationMatrix(np.array([[1, 0, 0], [0, -1, 0], [0, 0, -1]]))
R_worldiiwa0 = RotationMatrix(np.array([[0, -1, 0], [1, 0, 0], [0, 0, 1]]))
R_iiwa0world = R_worldiiwa0.inverse()
R_iiwa0wsg0 = R_iiwa0world.multiply(R_worldwsg0)

X_iiwa0world = RigidTransform(R_iiwa0world, p_iiwa0world)
X_worldiiwa0 = X_iiwa0world.inverse()

#iiwa0 to iiwa1 rotations
R_iiwa0iiwa1 = RotationMatrix.MakeZRotation(np.pi)

#iiwa1 transforms
p_iiwa1world = [0, -0.5, 0]
p_wsg1eraser = np.array([0, 0.0625, 0])
p_erasertable = np.array([0, 0, 0.16])

R_worldwsg1 = RotationMatrix(np.array([[0, 0, 1], [-1, 0, 0], [0, -1, 0]]))
R_worldiiwa1 = R_worldiiwa0.multiply(R_iiwa0iiwa1)
R_iiwa1world = R_worldiiwa1.inverse()
R_iiwa1wsg1 = R_iiwa1world.multiply(R_worldwsg1)

X_iiwa1world = RigidTransform(R_iiwa1world, p_iiwa1world)
X_worldiiwa1 = X_iiwa1world.inverse()

class PoseTrajectorySource(LeafSystem):
    def __init__(self, pose_trajectory):
        LeafSystem.__init__(self)
        self._pose_trajectory = pose_trajectory
        self.DeclareAbstractOutputPort(
            "pose", lambda: AbstractValue.Make(RigidTransform()),
            self.CalcPose)

    def CalcPose(self, context, output):
        output.set_value(self._pose_trajectory.GetPose(context.get_time()))

num_key_frames = 15
traj_time = 15
iiwa1_delay_time = 10

def create_smiley(num_key_frames, traj_time, iiwa1_delay_time):

    #choose dimensions
    eye_size = np.random.uniform(0.05,0.1)
    eye_distance = np.random.uniform(0.2,0.3)
    smile_type = np.random.randint(0,2)
    smile_size = np.random.uniform(0.1,0.3)
    lower_smile_size = np.random.uniform(smile_size/2,3*(smile_size/2))
    upper_smile_size = np.random.uniform(lower_smile_size + 0.1,5*(smile_size/2))

    upper_theta = abs(np.arcsin(smile_size/2/upper_smile_size))
    lower_theta = abs(np.arcsin(smile_size/2/lower_smile_size))

    if smile_type == 0:
        eye_thetas = np.linspace(0, 2*np.pi, int(num_key_frames/4)-1)
        eye_thetas = np.append(eye_thetas,[2*np.pi])
        upper_thetas = np.linspace(-1*upper_theta, upper_theta, int(num_key_frames/4))
        lower_thetas = np.linspace(lower_theta, -1*lower_theta, int(num_key_frames/4))
    else:
        eye_thetas = np.linspace(0, 2*np.pi, int(num_key_frames/4)-1)
        eye_thetas = np.append(eye_thetas,[2*np.pi])
        upper_thetas = np.linspace(-1*upper_theta, upper_theta, int(num_key_frames/2))

    #eyes
    eyes =[]
    for i in range(2):
        for theta in eye_thetas:
            eyes.append(np.array([((-1)**i)*eye_distance/2,0]).T + 
                np.array([eye_size*-1*np.cos(theta), eye_size*-1*np.sin(theta)]).T)

    #mouth
    mouth = []
    ##upper
    upper_center = np.array([0, -1*(eye_size+0.05)+abs(upper_smile_size*np.cos(upper_theta))]).T
    for theta in upper_thetas:
        mouth.append(upper_center + 
            np.array([upper_smile_size*np.sin(theta), -1*(upper_smile_size*np.cos(theta))]).T)

    ##lower
    if smile_type == 0:
        lower_center = np.array([0, -1*(eye_size+0.05)+abs(lower_smile_size*np.cos(lower_theta))]).T
        for theta in lower_thetas:
            mouth.append(lower_center + 
                np.array([lower_smile_size*np.sin(theta), -1*(lower_smile_size*np.cos(theta))]).T)

    _2D_points = np.concatenate((eyes, mouth), axis=0).T

    for i in range(_2D_points.shape[1]):
        _2D_points[:,i] = np.array([_2D_points[1,i] + eye_size, _2D_points[0,i]- eye_size/2]).T

    return smile_type, _2D_points


def points_in_world(_2D_points):
    traj = []
    _3D_points = np.vstack((_2D_points,np.ones(_2D_points.shape[1])*0.21))

    for i in range(_3D_points.shape[1]):
        traj.append(RigidTransform(np.array(_3D_points[:,i]).T))

    return traj


def get_X_WG(diagram, station, name):

    context = diagram.CreateDefaultContext()
    plant = station.GetSubsystemByName("plant")
    plant_context = diagram.GetMutableSubsystemContext(plant, context)
    station_context = diagram.GetMutableSubsystemContext(station, context)

    # provide initial states
    q0 = np.array([ 1.40666193e-05,  1.56461165e-01, -3.82761069e-05,
                       -1.32296976e+00, -6.29097287e-06,  1.61181157e+00, -2.66900985e-05])
    # set the joint positions of the kuka arm
    if name == 'iiwa0':
        iiwa = plant.GetModelInstanceByName("iiwa0")
        wsg = plant.GetModelInstanceByName("wsg0")
        gripper_frame = plant.GetFrameByName('body',station.GetSystems()[0].GetModelInstanceByName('wsg0'))
    
    if name == 'iiwa1':
        iiwa = plant.GetModelInstanceByName("iiwa1")
        wsg = plant.GetModelInstanceByName("wsg1")
        gripper_frame = plant.GetFrameByName('body',station.GetSystems()[0].GetModelInstanceByName('wsg1'))

    plant.SetPositions(plant_context, iiwa, q0)
    plant.SetVelocities(plant_context, iiwa, np.zeros(7))
        
    plant.SetPositions(plant_context, wsg, [-0.05, 0.05])
    plant.SetVelocities(plant_context, wsg, [0, 0])
    
    world_frame = plant.world_frame()

    X_WG = plant.CalcRelativeTransform(
        plant_context,
        frame_A=world_frame,
        frame_B=gripper_frame)

    print(X_WG)
    return X_WG

def visualize_key_frames(frame_poses):
    for i, pose in enumerate(frame_poses):
        AddMeshcatTriad(meshcat, 'frame_{}'.format(i),
                        length=0.02, radius=0.006, X_PT=pose)

def compose_key_frames(_2D_points, smile_type, name, X_WorldGripper_init):

    _3D_points = np.vstack((_2D_points,np.ones(_2D_points.shape[1])*0.2))

    predraw_pose_1 = make_predraw_pose(_3D_points[:,0], name)

    eyes = 0

    if name == 'iiwa0':
        key_frame_poses_in_robot = [X_iiwa0world.multiply(X_WorldGripper_init)]
        key_frame_poses_in_robot.append(X_iiwa0world.multiply(predraw_pose_1))
        key_frame_poses_in_robot.append(X_iiwa0world.multiply(predraw_pose_1))

        for i in range(_3D_points.shape[1]):
            pose = (RigidTransform(R_worldwsg0, p_wsg0chalk + p_chalktable + _3D_points[:,i].T))
            key_frame_poses_in_robot.append(X_iiwa0world.multiply(pose))

            if smile_type == 0 and ((i+1)% int(_3D_points.shape[1]/4) == 0):
                predraw_pose_1 = make_predraw_pose(_3D_points[:,i-1], name)
                key_frame_poses_in_robot.append(X_iiwa0world.multiply(predraw_pose_1))
                key_frame_poses_in_robot.append(X_iiwa0world.multiply(predraw_pose_1))
                predraw_pose_1 = make_predraw_pose(_3D_points[:,i], name)
                key_frame_poses_in_robot.append(X_iiwa0world.multiply(predraw_pose_1))
                key_frame_poses_in_robot.append(X_iiwa0world.multiply(predraw_pose_1))

            elif (smile_type == 1) and ((((i+1)% int(_3D_points.shape[1]/2)) == 0) or 
                ((((i+1)% int(_3D_points.shape[1]/4)) == 0) and (eyes == 0))):
                eyes = 1
                predraw_pose_1 = make_predraw_pose(_3D_points[:,i-1], name)
                key_frame_poses_in_robot.append(X_iiwa0world.multiply(predraw_pose_1))
                key_frame_poses_in_robot.append(X_iiwa0world.multiply(predraw_pose_1))
                predraw_pose_1 = make_predraw_pose(_3D_points[:,i], name)
                key_frame_poses_in_robot.append(X_iiwa0world.multiply(predraw_pose_1))
                key_frame_poses_in_robot.append(X_iiwa0world.multiply(predraw_pose_1))

        key_frame_poses_in_robot.append(X_iiwa0world.multiply(X_WorldGripper_init))

    elif name == 'iiwa1':
        key_frame_poses_in_robot = [X_iiwa1world.multiply(X_WorldGripper_init)]
        key_frame_poses_in_robot.append(X_iiwa1world.multiply(predraw_pose_1))
        key_frame_poses_in_robot.append(X_iiwa1world.multiply(predraw_pose_1))

        for i in range(_3D_points.shape[1]):
            pose = (RigidTransform(R_worldwsg1, p_wsg1eraser + p_erasertable + _3D_points[:,i].T))
            key_frame_poses_in_robot.append(X_iiwa1world.multiply(pose))

            if smile_type == 0 and ((i+1)% int(_3D_points.shape[1]/4) == 0):
                predraw_pose_1 = make_predraw_pose(_3D_points[:,i-1], name)
                key_frame_poses_in_robot.append(X_iiwa1world.multiply(predraw_pose_1))
                key_frame_poses_in_robot.append(X_iiwa1world.multiply(predraw_pose_1))
                predraw_pose_1 = make_predraw_pose(_3D_points[:,i], name)
                key_frame_poses_in_robot.append(X_iiwa1world.multiply(predraw_pose_1))
                key_frame_poses_in_robot.append(X_iiwa1world.multiply(predraw_pose_1))

            elif (smile_type == 1) and ((((i+1)% int(_3D_points.shape[1]/2)) == 0) or 
                ((((i+1)% int(_3D_points.shape[1]/4)) == 0) and (eyes == 0))):
                eyes = 1
                predraw_pose_1 = make_predraw_pose(_3D_points[:,i-1], name)
                key_frame_poses_in_robot.append(X_iiwa1world.multiply(predraw_pose_1))
                key_frame_poses_in_robot.append(X_iiwa1world.multiply(predraw_pose_1))
                predraw_pose_1 = make_predraw_pose(_3D_points[:,i], name)
                key_frame_poses_in_robot.append(X_iiwa1world.multiply(predraw_pose_1))
                key_frame_poses_in_robot.append(X_iiwa1world.multiply(predraw_pose_1))

        key_frame_poses_in_robot.append(X_iiwa1world.multiply(X_WorldGripper_init))

    return key_frame_poses_in_robot

def make_predraw_pose(pose,name):
    if name == 'iiwa0':
        predraw_pose_1 = RigidTransform(R_worldwsg0, (p_wsg0chalk + p_chalktable + pose.T + [0,0,0.2]))
    elif name == 'iiwa1':
        predraw_pose_1 = RigidTransform(R_worldwsg1,(p_wsg1eraser + p_erasertable + pose.T + [0,0,0.2]))
    return predraw_pose_1

In [7]:
diagram, station = MescatViz(meshcat)

In [8]:
num_key_frames = 80 #must be multiple of 4
traj_time = 80
iiwa1_delay_time = 80

smile_type, smile_2D = create_smiley(num_key_frames, traj_time, iiwa1_delay_time)

X_Worldwsg0_init = get_X_WG(diagram, station, 'iiwa0')
X_Worldwsg1_init = get_X_WG(diagram, station, 'iiwa1')

iiwa0_key_frame_poses = compose_key_frames(smile_2D, smile_type, 'iiwa0', X_Worldwsg0_init)
iiwa1_key_frame_poses = compose_key_frames(smile_2D, smile_type, 'iiwa1', X_Worldwsg1_init)

iiwa0_key_frame_poses_in_world = []
iiwa1_key_frame_poses_in_world = []

for frame in iiwa0_key_frame_poses:
    iiwa0_key_frame_poses_in_world.append(X_worldiiwa0.multiply(frame))
for frame in iiwa1_key_frame_poses:
    iiwa1_key_frame_poses_in_world.append(X_worldiiwa1.multiply(frame))

#visualize_key_frames(np.concatenate((iiwa0_key_frame_poses_in_world,iiwa1_key_frame_poses_in_world, 
#    points_in_world(smile_2D)), axis=None))
#visualize_key_frames(points_in_world(smile_2D))

if smile_type == 0:
    extra_frames = 16 + 4
else:
    extra_frames = 12 + 4

iiwa0_times = np.linspace(0, traj_time, num_key_frames + extra_frames)
iiwa1_times = np.linspace(iiwa1_delay_time, traj_time + iiwa1_delay_time, num_key_frames + extra_frames)

iiwa0_traj = PiecewisePose.MakeLinear(iiwa0_times, iiwa0_key_frame_poses)
iiwa1_traj = PiecewisePose.MakeLinear(iiwa1_times, iiwa1_key_frame_poses)


RigidTransform(
  R=RotationMatrix([
    [-0.9999999999051579, 1.3437349077363588e-05, 3.0204696644648582e-06],
    [-2.340354995851716e-06, 0.05032888698822864, -0.9987326985380272],
    [-1.3572336781715093e-05, -0.9987326984503733, -0.05032888695200715],
  ]),
  p=[-0.499989356653449, 0.4686663152509269, 0.6012858601360346],
)
RigidTransform(
  R=RotationMatrix([
    [0.9999999999051579, -1.3437349077357423e-05, -3.0204696645871664e-06],
    [2.34035499597418e-06, -0.05032888698822864, 0.9987326985380272],
    [-1.3572336781715093e-05, -0.9987326984503733, -0.05032888695200715],
  ]),
  p=[0.49998935665344907, -0.4686663152509269, 0.6012858601360346],
)


In [9]:
diagram, station = MescatViz(meshcat, iiwa0_traj, iiwa1_traj)

In [10]:
builder = DiagramBuilder()
traj1 = PiecewisePolynomial.FirstOrderHold([0, 1], [[1, 2]])
trajectory_source = builder.AddSystem(MyTrajectorySource(traj1))
diagram = builder.Build()
simulator = Simulator(diagram)

simulator.AdvanceTo(traj1.end_time())

# Note: next trajectory starts at traj1.end_time()
traj2 = PiecewisePolynomial.FirstOrderHold([1, 2], [[3, 4]])
trajectory_source._trajectory = traj2

simulator.AdvanceTo(traj2.end_time())

NameError: name 'MyTrajectorySource' is not defined

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=ba8e65c7-ba08-42d3-b249-70b2fa081e19' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>